The third - after increasing dropouts!

In [ ]:
!pip3 install -q transformers datasets

In [ ]:
import os
import json
import math
import glob
import torch
import shutil
import traceback

import numpy as np
import pandas as pd

from copy import deepcopy
from datasets import Dataset
from datetime import datetime
from sklearn.utils import shuffle
from transformers import EvalPrediction
from datasets.dataset_dict import DatasetDict
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score, classification_report
from transformers import AutoTokenizer, DataCollatorWithPadding, AutoModelForSequenceClassification, TrainingArguments, Trainer

In [ ]:
data = pd.read_excel("intermediate_data.xlsx")
final_data = pd.read_excel("final_data.xlsx")

In [ ]:
labels = ["Val_0", "Val_1"]
report_folder_name = "Reports"

In [ ]:
if not os.path.isdir(report_folder_name):
  os.mkdir(report_folder_name)
else:
  shutil.rmtree(report_folder_name)
  os.mkdir(report_folder_name)

In [ ]:
def preprocess_dataframe(data_df, convert_into_binary=True):
  if convert_into_binary:
    data_df = pd.concat((data_df, pd.get_dummies(data_df["memory_change"])), axis=1)
    data_df = data_df.rename(columns={"file_content": "text", 0:"Val_0", 1:"Val_1"})
    data_df.drop(list(set(data_df.columns).difference(set(["text", "Val_0", "Val_1"]))), inplace=True, axis=1)      
    data_df = data_df.replace({0:False, 1: True})
  else:
    data_df = data_df.rename(columns={"file_content": "text", "memory_change": "actual"})
    data_df.drop(list(set(data_df.columns).difference(set(["text", "actual"]))), inplace=True, axis=1)
  data_df.reset_index(drop=True, inplace=True)
  return data_df

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

In [ ]:
def preprocess_data(examples):
  # take a batch of texts
  text = examples["text"]
  # encode them
  encoding = tokenizer(text)
  # add labels
  labels_batch = {k: examples[k] for k in examples.keys() if k in labels}
  # create numpy array of shape (batch_size, num_labels)
  labels_matrix = np.zeros((len(text), len(labels)))
  # fill numpy array
  for idx, label in enumerate(labels):
    labels_matrix[:, idx] = labels_batch[label]

  encoding["labels"] = labels_matrix.tolist()
  
  return encoding

In [ ]:
def multi_label_metrics(predictions, labels, threshold=0.5):
    # first, apply sigmoid on predictions which are of shape (batch_size, num_labels)
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(torch.Tensor(predictions))
    # next, use threshold to turn them into integer predictions
    y_pred = np.zeros(probs.shape)
    y_pred[np.where(probs >= threshold)] = 1
    # finally, compute metrics
    y_true = labels
    f1_micro_average = f1_score(y_true=y_true, y_pred=y_pred, average='micro')
    roc_auc = roc_auc_score(y_true, y_pred, average = 'micro')
    accuracy = accuracy_score(y_true, y_pred)
    # return as dictionary
    metrics = {'f1': f1_micro_average,
               'roc_auc': roc_auc,
               'accuracy': accuracy}
    return metrics

def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions, 
            tuple) else p.predictions
    result = multi_label_metrics(
        predictions=preds, 
        labels=p.label_ids)
    return result

In [ ]:
def get_classification_report_row(classification_report):
    data_dict = {
                    "Precision": classification_report["weighted avg"]["precision"],
                    "Recall": classification_report["weighted avg"]["recall"],
                    "f1-score": classification_report["weighted avg"]["f1-score"],
                    "support": classification_report["weighted avg"]["support"],
                    "Accuracy": classification_report["accuracy"]
                }
    return data_dict

In [ ]:
data

,mapped_user_id,memory_change,PALFAMS28Percentile
0,2188,0,53.0
1,1338,0,50.0
2,1538,0,46.0
3,1717,0,72.0
4,2035,0,26.0
...,...,...,...
1805,3126,0,NaN
1806,1883,0,NaN
1807,3147,0,NaN
1808,2213,0,NaN


In [ ]:
# final_data = final_data[final_data.user_id.isin(list(data["mapped_user_id"]))]
# final_data_1 = final_data[final_data.memory_change==1]
# final_data_0 = final_data[final_data.memory_change==0]
# final_data_0 = final_data_0.sample(n = len(final_data_1), random_state = 42)
# final_data = pd.concat([final_data_0, final_data_1], ignore_index=True)
# data = data[data.mapped_user_id.isin(list(final_data["user_id"]))]
# data = shuffle(data, random_state=42)
# final_data = shuffle(final_data, random_state=42)

In [ ]:
# final_data.memory_change.value_counts()

In [ ]:
# data.memory_change.value_counts()

In [ ]:
# data

In [ ]:
k = 10
batch_size = 16
train_epochs = 5
metric_name = "f1"
final_stat = []
output_folder_name = f"bert-finetuned-by-chanpreet"
probability_threshold = 0.5

skf = StratifiedKFold(n_splits=k)

for i, (train_index, test_index) in enumerate(skf.split(data["mapped_user_id"], data["memory_change"])):
    train_data_user_id = list(data.iloc[train_index]["mapped_user_id"])
    train_data = final_data[final_data.user_id.isin(train_data_user_id)]
    train_data = preprocess_dataframe(train_data)
    train_data = Dataset.from_pandas(train_data)

    test_data_user_id = list(data.iloc[test_index]["mapped_user_id"])
    test_data = final_data[final_data.user_id.isin(test_data_user_id)]
    test_data_copy = deepcopy(test_data)
    test_data = preprocess_dataframe(test_data)
    test_data_copy = preprocess_dataframe(test_data_copy, convert_into_binary=False)
    test_data = Dataset.from_pandas(test_data)

    print("Fold {2}: Train size: {0}, Test size: {1}, Test range: {3}-{4}".format(len(train_data), len(test_data), i+1, min(test_index), max(test_index)))
    # print(test_index)

    train_dataset, validation_dataset = train_data.train_test_split(test_size=0.1).values()
    dataset = DatasetDict({"train": train_dataset,
                          "validation": validation_dataset,
                          "test": test_data
                           })
    encoded_dataset = dataset.map(preprocess_data, batched=True, remove_columns=dataset["train"].column_names)

    labels = [label for label in dataset['train'].features.keys() if label not in ["text"]]
    id2label = {idx:label for idx, label in enumerate(labels)}
    label2id = {label:idx for idx, label in enumerate(labels)}

    encoded_dataset.set_format("torch")

    model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased",
                                                               num_labels=len(labels),
                                                               id2label=id2label,
                                                               label2id=label2id,
                                                               attention_probs_dropout_prob=0.6, # by default it is 0.1
                                                              #  hidden_dropout_prob=0.7
                                                               )
    # print(model.config)
    # import pdb; pdb.set_trace()

    args = TrainingArguments(output_folder_name,
                             evaluation_strategy = "epoch",
                             save_strategy = "epoch",
                             learning_rate=2e-5,
                             per_device_train_batch_size=batch_size,
                             per_device_eval_batch_size=batch_size,
                             num_train_epochs=train_epochs,
                             weight_decay=0.01,
                             load_best_model_at_end=True,
                             metric_for_best_model=metric_name,
                             #push_to_hub=True,
                             )
    
    trainer = Trainer(model,
                      args,
                      train_dataset=encoded_dataset["train"],
                      eval_dataset=encoded_dataset["validation"],
                      tokenizer=tokenizer,
                      compute_metrics=compute_metrics
                      )
    
    trainer.train()
    print(json.dumps(trainer.evaluate(), indent=2))

    stat_list = []
    for ind, row in test_data_copy.iterrows():
        text = row["text"]
        encoding = tokenizer(text, return_tensors="pt")
        encoding = {k: v.to(trainer.model.device) for k,v in encoding.items()}
        outputs = trainer.model(**encoding)
        logits = outputs.logits
        sigmoid = torch.nn.Sigmoid()
        probs = sigmoid(logits.squeeze().cpu())
        predictions = np.zeros(probs.shape)
        predictions[np.where(probs >= probability_threshold)] = 1
        # turn predicted id's into actual label names
        predicted_labels = [id2label[idx] for idx, label in enumerate(predictions) if label == 1.0]
        if len(predicted_labels)>0:
          stat_list.append({"text": text,
                            "actual": row["actual"],
                            "predicted": int(predicted_labels[0].strip("Val_"))
                            })
    stat_df = pd.DataFrame(stat_list)
    stat_df.to_excel("{1}/{0}_fold_report.xlsx".format(i+1, report_folder_name), index=False, header=True)
    final_stat_dict = {"k": i+1,
                       "test_range": "{0}-{1}".format(min(test_index), max(test_index))
                      }
    final_stat_dict.update(get_classification_report_row(classification_report(stat_df["actual"], stat_df["predicted"], output_dict=True)))
    print(json.dumps(final_stat_dict, indent=2))
    final_stat.append(final_stat_dict)
    print("\n"*5)

    for x in sorted(glob.glob(os.path.join(output_folder_name, '*/')), key=os.path.getmtime)[1:]:
        try:
            shutil.rmtree(x)
        except OSError as e:
            print("Error: %s - %s." % (e.filename, e.strerror))


final_stat_df = pd.DataFrame(final_stat)
final_stat_df.to_excel("{0}/consolidated_report_{1}.xlsx".format(report_folder_name, datetime.now().strftime("%Y-%m-%d_%H:%M")), index=False, header=True)

Fold 1: Train size: 2083, Test size: 231, Test range: 0-185


Map:   0%|          | 0/1874 [00:00<?, ? examples/s]

Map:   0%|          | 0/209 [00:00<?, ? examples/s]

Map:   0%|          | 0/231 [00:00<?, ? examples/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

...

Map:   0%|          | 0/1881 [00:00<?, ? examples/s]

Map:   0%|          | 0/210 [00:00<?, ? examples/s]

Map:   0%|          | 0/223 [00:00<?, ? examples/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

Epoch,Training Loss,Validation Loss,F1,Roc Auc,Accuracy
1,No log,0.476492,0.823810,0.823810,0.823810
2,No log,0.480972,0.823810,0.823810,0.823810
3,0.426600,0.476375,0.823810,0.823810,0.823810
4,0.426600,0.484037,0.823810,0.823810,0.823810
5,0.424600,0.480691,0.823810,0.823810,0.823810
6,0.424600,0.513830,0.823810,0.823810,0.823810
7,0.410700,0.536607,0.823810,0.823810,0.823810
